## Sonification using the miditime library from Reveal
* Tutorial from: https://github.com/cirlabs/miditime
* Install miditime: pip3 install miditime 
* (*remember to use pip3 not pip, if you use python3*)


In [10]:
# LIBS
from miditime.miditime import MIDITime
import datetime

Instantiate the class with a tempo (120bpm is the default), an output file destination, the number of seconds you want to represent a year in the final song (default is 5 sec/year), the base octave (C5 is middle C, so the default is 5, and how many octaves you want your output to range over (default is 1).

In [9]:
mymidi = MIDITime(120, 'myfile.mid', 5, 5, 1) 

Bring in some data (this is some earthquakes). I'm assuming your data is already in date order, from oldest to newest.


In [17]:
# MADE UP DATA
my_data = [
    {'event_date': datetime.datetime(2017,2,2), 'magnitude': 3.4},
    {'event_date': datetime.datetime(2017,3,1), 'magnitude': 3.2},
    {'event_date': datetime.datetime(2017,3,30), 'magnitude': 3.6},
    {'event_date': datetime.datetime(2017,4,25), 'magnitude': 3.0},
    {'event_date': datetime.datetime(2017,4,30), 'magnitude': 5.6},
    {'event_date': datetime.datetime(2017,5,1), 'magnitude': 4.0}
]

Convert your date/time data into an integer, like days since the epoch (Jan. 1, 1970). You can use the days_since_epoch() helper method, or not:

In [18]:
my_data_epoched = [{'days_since_epoch': mymidi.days_since_epoch(d['event_date']), 'magnitude': d['magnitude']} for d in my_data] 

Convert your integer date/time to something reasonable for a song. For example, at 120 beats per minute, you'll need to scale the data down a lot to avoid a very long song if your data spans years. This uses the seconds_per_year attribute you set at the top, so if your date is converted to something other than days you may need to do your own conversion. But if your dataset spans years and your dates are in days (with fractions is fine), use the beat() helper method.

In [19]:
my_data_timed = [{'beat': mymidi.beat(d['days_since_epoch']), 'magnitude': d['magnitude']} for d in my_data_epoched]

In [20]:
start_time = my_data_timed[0]['beat']

Set up some functions to scale your other variable (magnitude in our case) to match your desired mode/key and octave range. There are helper methods to assist this scaling, very similar to a charting library like D3. You can choose a linear or logarithmic scale.

In [21]:
def mag_to_pitch_tuned(magnitude):
    # Where does this data point sit in the domain of your data? (I.E. the min magnitude is 3, the max in 5.6). In this case the optional 'True' means the scale is reversed, so the highest value will return the lowest percentage.
    scale_pct = mymidi.linear_scale_pct(3, 5.7, magnitude)

    # Another option: Linear scale, reverse order
    # scale_pct = mymidi.linear_scale_pct(3, 5.7, magnitude, True)

    # Another option: Logarithmic scale, reverse order
    # scale_pct = mymidi.log_scale_pct(3, 5.7, magnitude, True)

    # Pick a range of notes. This allows you to play in a key.
    c_major = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

    #Find the note that matches your data point
    note = mymidi.scale_to_note(scale_pct, c_major)

    #Translate that note to a MIDI pitch
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

In [22]:
note_list = []

for d in my_data_timed:
    note_list.append([
        d['beat'] - start_time,
        mag_to_pitch_tuned(d['magnitude']),
        100,  # velocity
        1  # duration, in beats
    ])

In [23]:
# Add a track with those notes
mymidi.add_track(note_list)

# Output the .mid file
mymidi.save_midi()

62 0.0 1 100
60 0.7400000000000091 1 100
62 1.5400000000000205 1 100
60 2.25 1 100
71 2.3799999999999955 1 100
64 2.410000000000025 1 100
